In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from datasets import load_dataset
from trl import SFTTrainer
import torch

# base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
base_model = "NousResearch/Llama-2-7b-chat-hf"
revision = "refs/pr/4"

quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=False)

# tokenizer = AutoTokenizer.from_pretrained(base_model, revision=revision)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# if tokenizer.pad_token_id is None:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
    
compute_dtype = getattr(torch, "float16")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

C:\Users\ryanw\anaconda3\envs\llama3-test\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ryanw\.cache\huggingface\hub\models--NousResearch--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [2]:

# model = AutoModelForCausalLM.from_pretrained(base_model, revision=revision, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=quant_config)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", torch_dtype=compute_dtype, quantization_config=quant_config, low_cpu_mem_usage=True, temperature=0.1, do_sample=True)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

C:\Users\ryanw\anaconda3\envs\llama3-test\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\Users\ryanw\anaconda3\envs\llama3-test\Lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\Users\ryanw\anaconda3\envs\llama3

In [3]:
# def generate_response(prompt):
#     chat = [{'content': prompt, 'role': 'user'}]
#     chat_tokens = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors='pt').to(model.device)
# 
#     new_chat_tokens = model.generate(chat_tokens, do_sample=False, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id)
#     new_chat_str = tokenizer.decode(new_chat_tokens[0])
#     return new_chat_str

In [4]:
# print(generate_response('''Think step by step and then provide a two or three sentence summary of what the code is doing for an audience who may not be familiar with machine learning. Focus on the problem the authors' are trying to solve and do nothing else. 
# 
# chunksize = 100000
# df_chunks = []
# 
# for df in pd.read_csv(
#     crimes_raw_file_path, parse_dates=[2], index_col=[2],
#     chunksize=chunksize, iterator=True, low_memory=True
# ):
#     df = df['2016-01-01':'2016-12-31']
#     df_chunks.append(df)
#     
# crimes2016 = pd.concat(df_chunks)
# 
# # Serialising & compressing the data
# crimes2016.sort_index(ascending=True).to_csv(crimes_file_path)
# 
# crimes2016.head()'''))

In [5]:
dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")

In [7]:
dataset = dataset.select(range(5))

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 5
})

In [9]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [10]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [11]:
trainer.train()

C:\Users\ryanw\anaconda3\envs\llama3-test\Lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss


TrainOutput(global_step=2, training_loss=2.1906633377075195, metrics={'train_runtime': 838.3595, 'train_samples_per_second': 0.006, 'train_steps_per_second': 0.002, 'total_flos': 178587030306816.0, 'train_loss': 2.1906633377075195, 'epoch': 1.0})

In [12]:
new_model = "llama-2-7b-chat-guanaco-testing"

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


('llama-2-7b-chat-guanaco-testing\\tokenizer_config.json',
 'llama-2-7b-chat-guanaco-testing\\special_tokens_map.json',
 'llama-2-7b-chat-guanaco-testing\\tokenizer.model',
 'llama-2-7b-chat-guanaco-testing\\added_tokens.json',
 'llama-2-7b-chat-guanaco-testing\\tokenizer.json')